In [1]:
# Imports

import numpy as np
from enum import Enum
from collections import deque
from queue import PriorityQueue
from queue import Queue
from copy import copy, deepcopy
import sys
import random
import math

In [2]:
# Coordinate Class
# Source: http://pythonfiddle.com/coordinate-class/

class Coordinate(object):
    
    def __init__(self,x,y):
        self.x = x
        self.y = y

    def getX(self):
        return self.x
    def getY(self):
        return self.y
    def getCoordinates(self):
        return self.x, self.y

    def __str__(self):
        return '<' + str(self.getX()) + ',' + str(self.getY()) + '>'  
    def __unicode__(self):
        return '<' + str(self.getX()) + ',' + str(self.getY()) + '>'
    def __repr__(self):
        return '<' + str(self.getX()) + ',' + str(self.getY()) + '>'
    
    def __eq__(self, other):
        return self.y == other.y and self.x == other.x
    
    # https://stackoverflow.com/questions/9135759/java-hashcode-for-a-point-class
    # Needed for using Coordinate objects as keys in Dictionaries
    def __hash__(self):
        result = self.x;
        result = 31 * result + self.y;
        return result;

In [3]:
# RoomElement Enum

class RoomElement(Enum):
    EMPTY = 0
    WALL = 1
    DOOR = 2
    HAZARD = 3
    EXIT = 4

In [4]:
# Pick N elements from a list K elements long
def random_elements_of_list(list, N, K):
    newList = []
    indexes = random.sample(range(N), K)
    for index in indexes:
        newList.append(list[index])
    return newList

In [5]:
# Room Class

class Room():
    
    def __init__(self):
        self.map = np.array([[]])
        self.hazards = []
        self.originalMap = self.map
        
    def __init__(self, m):
        self.map = np.array(m)
        self.hazards = []
        self.originalMap = self.map
    
    # Get all Coordinates of a specific RoomElement ENUM type (f.e. RoomElement.HAZARD)
    def get_element_coordinates_of_map(self, roomElement):
        spots = []
        y = 0
        for row in self.map:
            x = 0
            for value in row:
                if value == roomElement.value:
                    coord = Coordinate(y, x)
                    spots.append(coord)
                x += 1   
            y += 1
        return spots
    
    # Return if hazards prevent a person from being able to exit the map
    def isExitable(self):
        room_empty_spots = self.get_element_coordinates_of_map(RoomElement.EMPTY)
        room_exit_spots = self.get_element_coordinates_of_map(RoomElement.EXIT)
        exitable = False
        for exit in room_exit_spots:
            possiblePaths = self.paths_to_a_goal(room_empty_spots[0], exit)
            if len(possiblePaths) != 0:
                exitable = True
        return exitable
            
        
    # Add a new hazard to the map
    def create_new_hazard(self):
        # do something to map
        room_empty_spots = self.get_element_coordinates_of_map(RoomElement.EMPTY)
        if len(room_empty_spots) < 1:
            return None
        room_hazard_spot = random.choice(room_empty_spots)
        self.map[room_hazard_spot.x][room_hazard_spot.y] = RoomElement.HAZARD.value
        if not self.isExitable:
            self.map[room_hazard_spot.x][room_hazard_spot.y] = RoomElement.EMPTY.value
            return
        self.hazards.append(Coordinate(room_hazard_spot.x, room_hazard_spot.y))
        
    # Grow an existing hazard by one if possible
    def expand_existing_hazard(self):
        room_hazard_spots = self.get_element_coordinates_of_map(RoomElement.HAZARD)
        if len(room_hazard_spots) < 1:
            return None
        room_hazard_spot = random_elements_of_list(room_hazard_spots, len(room_hazard_spots), 1)[0]
        
        successors = []
        successors.append(Coordinate(room_hazard_spot.x - 1, room_hazard_spot.y))
        successors.append(Coordinate(room_hazard_spot.x + 1, room_hazard_spot.y))
        successors.append(Coordinate(room_hazard_spot.x, room_hazard_spot.y - 1))
        successors.append(Coordinate(room_hazard_spot.x, room_hazard_spot.y + 1))
        
        possibleExpansionSpots = []
        
        for c in successors:
            if c.x < 0 or c.y < 0 or c.x >= self.map.shape[0] or c.y >= self.map.shape[1]:
                continue
            if self.map[c.x][c.y] == RoomElement.WALL.value:
                continue
            if self.map[c.x][c.y] == RoomElement.HAZARD.value:
                continue
            if self.map[c.x][c.y] == RoomElement.DOOR.value:
                continue
            if self.map[c.x][c.y] == RoomElement.EXIT.value:
                continue
            possibleExpansionSpots.append(c)
            
            
        #new_room_hazard_spot = random_elements_of_list(possibleExpansionSpots, len(possibleExpansionSpots), 1)[0]
        if (len(possibleExpansionSpots) == 0):
            return
        new_room_hazard_spot = random.choice(possibleExpansionSpots)
        self.map[new_room_hazard_spot.x][new_room_hazard_spot.y] = RoomElement.HAZARD.value 
        if not self.isExitable:
            self.map[room_hazard_spot.x][room_hazard_spot.y] = RoomElement.EMPTY.value
            return
        self.hazards.append(Coordinate(new_room_hazard_spot.x, new_room_hazard_spot.y))
        
    # Return copy of map for debugging purposes
    def return_current_map_copy(self):
        mapCopy = deepcopy(self.map)
        return mapCopy
    
    # Return copy of map with one marked spot for debugging purposes
    def print_current_map_spot(self, c: Coordinate):
        mapCopy = deepcopy(self.map)
        mapCopy[c.x][c.y] = 9999
        return mapCopy
    
    # Return copy of map with multiple marked spots for debugging purposes
    def print_current_map_spots(self, mapSpots):
        mapCopy = deepcopy(self.map)
        for spot in mapSpots:
            mapCopy[spot.x][spot.y] = 9999
        return mapCopy
    
    # Get all possible paths from one point in the map to another
    def paths_to_a_goal(self, start: Coordinate, end: Coordinate):
        stack = Queue()
        visited = set()
        possiblePlans = []
        actions = []
        
        stack.put((start, actions))
        
        while not stack.empty():
            tupleObject = stack.get()
            coord = tupleObject[0]
            if coord.getCoordinates() in visited:
                continue
            
            actionListSoFar = tupleObject[1]
            
            visited.add(coord);
            if coord == end:
                possiblePlans.append(actionListSoFar)
                continue
            
            successors = []
            successors.append(Coordinate(coord.x - 1, coord.y))
            successors.append(Coordinate(coord.x + 1, coord.y))
            successors.append(Coordinate(coord.x, coord.y - 1))
            successors.append(Coordinate(coord.x, coord.y + 1))
            
            for c in successors:
                if c.x < 0 or c.y < 0 or c.x >= self.map.shape[0] or c.y >= self.map.shape[1]:
                    continue
                if self.map[c.x][c.y] == RoomElement.WALL.value:
                    continue
                if self.map[c.x][c.y] == RoomElement.HAZARD.value:
                    continue
                    
                
                if (not c in visited):
                    newActions = actionListSoFar + [c]
                    stack.put((c, newActions))
        
        return possiblePlans
    
    # Determine shortest length path based on a list of potential paths
    def best_path(self, paths):
        bestDistance = sys.maxsize
        bestPath = []
        
        for path in paths:
            unitsMoved = len(path)
            if (unitsMoved < bestDistance):
                bestDistance = unitsMoved
                bestPath = path
        
        return bestPath
    
    # Revert map to original state, pre-hazard additions
    def reset_map(self):
        self.map = self.originalMap
        self.hazards = []

In [6]:
# Person Class
class Person():
    
    def __init__(self):
        self.currentCoordinate = Coordinate(0, 0)
        self.path = []
        self.index = 0
        self.exited = False
        self.goal = Coordinate(0, 0)
        
    def __init__(self, c, p):
        self.currentCoordinate = c
        self.index = 0
        self.path = p
        self.exited = False
        if (len(self.path) == 0):
            self.exited = True
            self.goal = self.currentCoordinate
        else:
            self.goal = p[-1]
        
    # Determine if a person has exited the map based on current state variables
    def hasExited(self):
        if self.index >= len(self.path):
            self.exited = False
        if len(self.path) == 0:
            self.exited = True
        if (self.currentCoordinate == self.goal):
            self.exited = True
        return self.exited
    
    def get_current_coordinate(self):
        return self.currentCoordinate
        
    # Return next spot in the person's path
    def next_spot_in_path(self):
        if (len(self.path) == 1):
            return self.path[0]
        if self.hasExited():
            return None
        if (self.index + 1 >= len(self.path)):
            return None
        nextSpot = self.path[self.index]
        return nextSpot
    
    # Move forward the person in their path and update corresponding state variables
    def move_to_next_spot_in_path(self):
        if self.index >= len(self.path) - 1:
            self.exited = True
        if self.path[-1] == self.currentCoordinate:
            self.exited = True
        if not self.hasExited():
            if len(self.path) == 1:
                self.currentCoordinate = self.path[0]
            else:
                self.currentCoordinate = self.path[self.index]
            self.index = self.index + 1

    def __str__(self):
        return 'Person: @' + str(self.currentCoordinate) + ', Path: ' + str(self.path) + ', Exited? ' + str(self.exited)  
    def __unicode__(self):
        return 'Person: @' + str(self.currentCoordinate) + ', Path: ' + str(self.path) + ', Exited? ' + str(self.exited)  
    def __repr__(self):
        return 'Person: @' + str(self.currentCoordinate) + ', Path: ' + str(self.path) + ', Exited? ' + str(self.exited)  

In [22]:
# EvacuationSimulation Class
class EvacuationSimulation():
    
    def __init__(self):
        self.room = Room()
        self.peopleCoordinates = []
        
    def __init__(self, r, p):
        self.room = r
        self.peopleCoordinates = p
        
    # Get person path based on current coordinate and room
    def get_path_based_on_current_coordinate(self, coordinate, room):
        paths = []
        exits = room.get_element_coordinates_of_map(RoomElement.EXIT)
        for exit in exits:
            somePaths = self.room.paths_to_a_goal(coordinate, exit)
            paths.extend(somePaths)
        bestPath = room.best_path(paths)
        return bestPath
    
    # Get sum of all people's initial paths for the room evacuation
    def pathsum_aggregate(self, personList):
        total = 0
        for person in personList:
            pathLength = len(person.path)
            total = total + pathLength
        return total
    
    # Boolean check for if every person has evacuated the room
    def everyone_has_exited(self, personList):
        for person in personList:
            if not person.hasExited():
                return False
        return True

    # Default evacuation method, doesn't deploy hazard creation
    def simulate(self):
        personList = []
        collisionCount = 0
        error = False
        for coordinate in self.peopleCoordinates:
            personPath = self.get_path_based_on_current_coordinate(coordinate, self.room)
            newPerson = Person(coordinate, personPath)
            personList.append(newPerson)
        pathsum = self.pathsum_aggregate(personList)
        
        print('Initialize simulation')
        time = 0
        print('Time: ' + str(time))
        print(self.room.print_current_map_spots(self.peopleCoordinates))
        #print(personList)
        
        # Start Loop   
        while not self.everyone_has_exited(personList):
            
            locationDict = {}
            for person in personList:
                if (person.next_spot_in_path() is None):
                    continue
                nextPlaceInPath = person.next_spot_in_path()
                #print("Next spot: " + str(nextPlaceInPath))
                
                if nextPlaceInPath in locationDict and locationDict[nextPlaceInPath] is not None and len(locationDict[nextPlaceInPath]) > 1:
                    # TODO: When > 0, issues! Need to fix to get proper collision logic... same for simulate_with_hazards()
                    personList = locationDict[nextPlaceInPath]
                    personList.append(person)
                    locationDict[nextPlaceInPath] = personList
                    print("Multiple people in same spot:" + str(locationDict[nextPlaceInPath]))
                else:
                    locationDict[nextPlaceInPath] = [person]
                    
            
            print("Dictionary: " + str(locationDict))
            if len(locationDict.keys()) == 0:
                break
                
            keyPopList = []
            for key in locationDict.keys():
                #print("Spot: " + str(key))
                #print("People: " + str(locationDict[key]))
                if locationDict[key] is None:
                    continue
                
                if len(locationDict[key]) == 1:
                    # person can move on
                    if locationDict[key][0].hasExited():
                        #locationDict.pop(key, None)
                        keyPopList.append(key)
                    else:
                        personToMove = locationDict[key][0]
                        personToMove.move_to_next_spot_in_path()
                else:
                    # one random person moves forward
                    print("Collision!")
                    persontoMove = random.choice(locationDict[key])
                    personToMove.move_to_next_spot_in_path()
                    collisionCount = collisionCount + 1
                    
            for key in keyPopList:
                locationDict.pop(key, None)
            
            
            
            time = time + 1
            print('Time: ' + str(time))
            coordinates = []
            for person in personList:
                coordinates.append(person.get_current_coordinate())
            print('Coordinates: ' + str(coordinates))
            print(self.room.print_current_map_spots(coordinates))
            
            if (time == 1000):
                print("ERROR encountered.")
                error = True
                break
            
            
        # End Loop
        stats_dict = {
            "time": time,
            "personCount": len(personList),
            "collisions": collisionCount,
            "pathSumAggregate": pathsum,
            "error": error,
            "map": self.room
        }
        
        print('Done with simulation')
        return stats_dict
        
        
        
    # Evacuation method that does employ hazard creation
    def simulate_with_hazards(self):
        personList = []
        collisionCount = 0
        deathCount = 0
        error = False
        for coordinate in self.peopleCoordinates:
            personPath = self.get_path_based_on_current_coordinate(coordinate, self.room)
            newPerson = Person(coordinate, personPath)
            personList.append(newPerson)
        pathsum = self.pathsum_aggregate(personList)
        
        print('Initialize simulation')
        time = 0
        print('Time: ' + str(time))
        print(self.room.print_current_map_spots(self.peopleCoordinates))
        #print(personList)
        
        # Start Loop 
        while not self.everyone_has_exited(personList):
            
            randomNumber = random.randint(0, 10)
            #print("Random Number: " + str(randomNumber))
            pathreevaluation = False
            
            if (randomNumber < 3):
                self.room.create_new_hazard()
                pathreevaluation = True
            if (randomNumber > 7):
                self.room.expand_existing_hazard()
                pathreevaluation = True
            print("Hazards: " + str(self.room.hazards))
            # People need to adjust pathing
            if pathreevaluation:
                for person in personList:
                    if person.hasExited():
                        continue
                    #print("Before: " + str(person.path))
                    person.path = self.get_path_based_on_current_coordinate(person.currentCoordinate, self.room)
                    #print("After: " + str(person.path))
                    person.index = 0
                    # TODO: what if path is length of 0? no possible path then... person dies??
                    if (len(person.path) == 0):
                        person.exited = True
                        deathCount = deathCount + 1
                    else:
                        person.currentCoordinate = person.path[0]
                        person.exited = False
                    
                
            locationDict = {}
            for person in personList:
                if (person.next_spot_in_path() is None):
                    continue
                nextPlaceInPath = person.next_spot_in_path()
                #print("Next spot: " + str(nextPlaceInPath))
                
                if nextPlaceInPath in locationDict and locationDict[nextPlaceInPath] is not None and len(locationDict[nextPlaceInPath]) > 1:
                    # TODO: When > 0, issues!
                    personList = locationDict[nextPlaceInPath]
                    personList.append(person)
                    locationDict[nextPlaceInPath] = personList
                    print("Multiple people in same spot:" + str(locationDict[nextPlaceInPath]))
                else:
                    locationDict[nextPlaceInPath] = [person]
                    
            
            print("Dictionary: " + str(locationDict))
            if len(locationDict.keys()) == 0:
                break
            
            for key in list(locationDict.keys()):
                #print("Spot: " + str(key))
                #print("People: " + str(locationDict[key]))
                people = locationDict[key]
                if len(people) == 1 and people[0].exited is True:
                    locationDict.pop(key, None)
                    
                allExited = True
                for person in people:
                    if person.exited is False:
                        allExited = False
                if allExited is True:
                    locationDict.pop(key, None)
                
            if len(locationDict.keys()) == 0:
                stats_dict = {
                    "time": time,
                    "personCount": len(personList),
                    "collisions": collisionCount,
                    "hazards": len(self.room.hazards),
                    "pathSumAggregate": pathsum,
                    "deaths": deathCount,
                    "error": error,
                    "map": self.room
                }
                return stats_dict
                
            
            for key in locationDict.keys():
                #print(key)
                #print(locationDict[key])
                if locationDict[key] is None:
                    continue
                
                if len(locationDict[key]) == 1:
                    # person can move on
                    #if locationDict[key][0].hasExited():
                        #locationDict.pop(key, None)
                    personToMove = locationDict[key][0]
                    personToMove.move_to_next_spot_in_path()
                else:
                    # one random person moves forward
                    print("Collision!")
                    #randomIndex = random.sample(range(len(locationDict[key])), 1)
                    #persontoMove = locationDict[key][randomIndex]
                    persontoMove = random.choice(locationDict[key])
                    personToMove.move_to_next_spot_in_path()
                    collisionCount = collisionCount + 1
                    
            time = time + 1
            print('Time: ' + str(time))
            coordinates = []
            for person in personList:
                coordinates.append(person.get_current_coordinate())
            print('Coordinates: ' + str(coordinates))
            print(self.room.print_current_map_spots(coordinates))
            
            if (time == 1000):
                print("ERROR encountered.")
                error = True
                break
            
            
        # End Loop
        print('Done with simulation')
        
        stats_dict = {
            "time": time,
            "personCount": len(personList),
            "collisions": collisionCount,
            "hazards": len(self.room.hazards),
            "pathSumAggregate": pathsum,
            "deaths": deathCount,
            "error": error,
            "map": self.room
        }
        return stats_dict

In [8]:
# ROOM 1
# One exit in a walled empty room.
room_1_map = [
    [1, 1, 1, 1, 1, 1, 1, 1, 4, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]
room_1 = Room(room_1_map)
room_1_empty_spots = room_1.get_element_coordinates_of_map(RoomElement.EMPTY)

room_1_empty_spots_subset = random_elements_of_list(room_1_empty_spots, len(room_1_empty_spots), int(len(room_1_empty_spots)/10))
simulation1 = EvacuationSimulation(room_1, room_1_empty_spots_subset)
simulation1.simulate()

Initialize simulation
Time: 0
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0 9999    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0 9999    0 9999    1]
 [   1    0 9999    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0 9999    0    0    0    0    0 9999    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    1    1    1    1    1    1    1    1    1]]
Dictionary: {<3,2>: [Person: @<4,2>, Path: [<3,2>, <2,2>, <1,2>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False], <2,8>: [Person: @<3,8>, Path: [<2,8>, <1,8>, <0,8>], Exited? False], <1,2>: [Person: @<2,2>, Path: [<1,2>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False], <6,2>: [Person: @<7,2>, Path: [<6,2>, <5,2>, <4,2>, <3,2>, <2,2>, <1,2>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? Fals

{'time': 12,
 'personCount': 6,
 'collisions': 0,
 'pathSumAggregate': 46,
 'error': False,
 'map': <__main__.Room at 0x7fc9e0438b20>}

In [9]:
simulation1.simulate_with_hazards()

Initialize simulation
Time: 0
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0 9999    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0 9999    0 9999    1]
 [   1    0 9999    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0 9999    0    0    0    0    0 9999    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    1    1    1    1    1    1    1    1    1]]
Hazards: [<8,1>]
Dictionary: {<3,2>: [Person: @<3,2>, Path: [<3,2>, <2,2>, <1,2>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False], <2,8>: [Person: @<2,8>, Path: [<2,8>, <1,8>, <0,8>], Exited? False], <1,2>: [Person: @<1,2>, Path: [<1,2>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False], <6,2>: [Person: @<6,2>, Path: [<6,2>, <5,2>, <4,2>, <3,2>, <2,2>, <1,2>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,

Dictionary: {<1,8>: [Person: @<1,8>, Path: [<1,8>, <0,8>], Exited? False], <0,8>: [Person: @<0,8>, Path: [<0,8>], Exited? True], <2,4>: [Person: @<2,4>, Path: [<2,4>, <2,5>, <2,6>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False]}
Time: 9
Coordinates: [<1,8>, <0,8>, <0,8>, <2,4>, <0,8>, <0,8>]
[[   1    1    1    1    1    1    1    1 9999    1]
 [   1    0    0    0    0    3    0    0 9999    1]
 [   1    0    0    0 9999    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    3    3    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    3    0    0    0    0    3    1]
 [   1    3    3    0    0    0    0    0    0    1]
 [   1    1    1    1    1    1    1    1    1    1]]
Hazards: [<8,1>, <8,2>, <4,6>, <4,7>, <1,5>, <7,8>, <7,3>, <1,3>]
Dictionary: {<0,8>: [Person: @<0,8>, Path: [<0,8>], Exited? True], <2,5>: [Person: @<2,5>, Path: [<2,5>, <2,6>, <

{'time': 14,
 'personCount': 6,
 'collisions': 0,
 'hazards': 12,
 'pathSumAggregate': 46,
 'deaths': 0,
 'error': False,
 'map': <__main__.Room at 0x7fc9e0438b20>}

In [10]:
# ROOM 2
# Same as room 1 but the walking path is a square.
room_2_map = [
    [1, 1, 1, 1, 1, 1, 1, 1, 4, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]
room_2 = Room(room_2_map)

room_2_empty_spots = room_2.get_element_coordinates_of_map(RoomElement.EMPTY)
room_2_empty_spots_subset = random_elements_of_list(room_2_empty_spots, len(room_2_empty_spots), int(len(room_2_empty_spots)/10))
print('Character Spots: ' + str(room_2_empty_spots_subset))

simulation2 = EvacuationSimulation(room_2, room_2_empty_spots_subset)
simulation2.simulate()

Character Spots: [<5,8>, <3,1>]
Initialize simulation
Time: 0
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    1    1    1    1    1    1    0    1]
 [   1 9999    1    1    1    1    1    1    0    1]
 [   1    0    1    1    1    1    1    1    0    1]
 [   1    0    1    1    1    1    1    1 9999    1]
 [   1    0    1    1    1    1    1    1    0    1]
 [   1    0    1    1    1    1    1    1    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    1    1    1    1    1    1    1    1    1]]
Dictionary: {<4,8>: [Person: @<5,8>, Path: [<4,8>, <3,8>, <2,8>, <1,8>, <0,8>], Exited? False], <2,1>: [Person: @<3,1>, Path: [<2,1>, <1,1>, <1,2>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False]}
Time: 1
Coordinates: [<4,8>, <2,1>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1 9999    1    1    1    1    1    1    0    1

{'time': 9,
 'personCount': 2,
 'collisions': 0,
 'pathSumAggregate': 15,
 'error': False,
 'map': <__main__.Room at 0x7fc9e04c22e0>}

In [11]:
simulation2.simulate_with_hazards()

Initialize simulation
Time: 0
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    1    1    1    1    1    1    0    1]
 [   1 9999    1    1    1    1    1    1    0    1]
 [   1    0    1    1    1    1    1    1    0    1]
 [   1    0    1    1    1    1    1    1 9999    1]
 [   1    0    1    1    1    1    1    1    0    1]
 [   1    0    1    1    1    1    1    1    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    1    1    1    1    1    1    1    1    1]]
Hazards: []
Dictionary: {<4,8>: [Person: @<5,8>, Path: [<4,8>, <3,8>, <2,8>, <1,8>, <0,8>], Exited? False], <2,1>: [Person: @<3,1>, Path: [<2,1>, <1,1>, <1,2>, <1,3>, <1,4>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False]}
Time: 1
Coordinates: [<4,8>, <2,1>]
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1 9999    1    1    1    1    1    1    0    1]
 [   1    0    1  

{'time': 9,
 'personCount': 2,
 'collisions': 0,
 'hazards': 2,
 'pathSumAggregate': 15,
 'deaths': 0,
 'error': False,
 'map': <__main__.Room at 0x7fc9e04c22e0>}

In [12]:
# ROOM 3
# Same as Room 1 but with two exits.
room_3_map = [
    [1, 1, 1, 1, 1, 1, 1, 1, 4, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 4, 1, 1, 1, 1, 1, 1, 1, 1]
]

room_3 = Room(room_3_map)

room_3_empty_spots = room_3.get_element_coordinates_of_map(RoomElement.EMPTY)
room_3_empty_spots_subset = random_elements_of_list(room_3_empty_spots, len(room_3_empty_spots), int(len(room_3_empty_spots)/10))
print('Character Spots: ' + str(room_3_empty_spots_subset))

simulation3 = EvacuationSimulation(room_3, room_3_empty_spots_subset)
simulation3.simulate()

Character Spots: [<8,8>, <4,1>, <3,5>, <7,1>, <8,7>, <7,4>]
Initialize simulation
Time: 0
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0 9999    0    0    0    1]
 [   1 9999    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1 9999    0    0 9999    0    0    0    0    1]
 [   1    0    0    0    0    0    0 9999 9999    1]
 [   1    4    1    1    1    1    1    1    1    1]]
Dictionary: {<7,8>: [Person: @<8,8>, Path: [<7,8>, <6,8>, <5,8>, <4,8>, <3,8>, <2,8>, <1,8>, <0,8>], Exited? False], <5,1>: [Person: @<4,1>, Path: [<5,1>, <6,1>, <7,1>, <8,1>, <9,1>], Exited? False], <2,5>: [Person: @<3,5>, Path: [<2,5>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False], <8,1>: [Person: @<7,1>, Path: [<8,1>, <9,1>], Exited? False], <8,6>: [Person: @<8,7>, Pat

{'time': 7,
 'personCount': 6,
 'collisions': 0,
 'pathSumAggregate': 33,
 'error': False,
 'map': <__main__.Room at 0x7fc9e0440310>}

In [13]:
simulation3.simulate_with_hazards()

Initialize simulation
Time: 0
[[   1    1    1    1    1    1    1    1    4    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0 9999    0    0    0    1]
 [   1 9999    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    0    1]
 [   1 9999    0    0 9999    0    0    0    0    1]
 [   1    0    0    0    0    0    0 9999 9999    1]
 [   1    4    1    1    1    1    1    1    1    1]]
Hazards: []
Dictionary: {<7,8>: [Person: @<8,8>, Path: [<7,8>, <6,8>, <5,8>, <4,8>, <3,8>, <2,8>, <1,8>, <0,8>], Exited? False], <5,1>: [Person: @<4,1>, Path: [<5,1>, <6,1>, <7,1>, <8,1>, <9,1>], Exited? False], <2,5>: [Person: @<3,5>, Path: [<2,5>, <1,5>, <1,6>, <1,7>, <1,8>, <0,8>], Exited? False], <8,1>: [Person: @<7,1>, Path: [<8,1>, <9,1>], Exited? False], <8,6>: [Person: @<8,7>, Path: [<8,6>, <8,5>, <8,4>, <8,3>, <8,2>, <8,1>, <9

{'time': 7,
 'personCount': 6,
 'collisions': 0,
 'hazards': 1,
 'pathSumAggregate': 33,
 'deaths': 0,
 'error': False,
 'map': <__main__.Room at 0x7fc9e0440310>}

In [14]:
# Randomize Map Method 1
'''
Randomly generate NxM array (25 > N > 5, 25 > M > 5) 
where the room is bounded by walls and all of the inner space is empty.
However, multiple doors will exist in the in the wall (6 > D > 1) (none of which isn't a corner).
'''
def generate_room_escape_doors(N, M, num_exits = None):
    room = []  
    for i in range(0, N):
        if i == 0 or i == N - 1:
            firstRow = list()
            for j in range(0, M):
                firstRow.append(1)
            room.append(firstRow)        
        else:
            for j in range(0, M):
                firstElement = 1
                lastElement = 1
                row = list()
                row.append(1)
                for k in range(0, M-2):
                    row.append(0)
                row.append(1)
            room.append(row)
    if num_exits is None:
        D = random.randint(1,7)
    else:
        D = num_exits
    for i in range(0, D):
        doorCoordinateX = random.randint(1, N-2)
        doorCoordinateY = random.choice([0, M-1])
        door2CoordinateX = random.choice([0, N-1])
        door2CoordinateY = random.randint(1, M-2)
        door = (doorCoordinateX, doorCoordinateY)
        door2 = (door2CoordinateX, door2CoordinateY)
        door_min = random.choice([door, door2])
        room[door_min[0]][door_min[1]] = 4
    return room

In [15]:
# Randomize Map Method 2
'''
Randomly generate NxM array (25 > N > 5, 25 > M > 5) 
where the room is bounded by walls and all of the inner space is empty.
However, one doors will exist in the in the wall that is multiple units long (consecutive D's in the wall),
and none of the D's can exist in any corner.
'''
def generate_room_consecutive_doors(N, M, num_exits = None):
    #N = random.randint(5,25)
    #M = random.randint(5,25)
    room = []
    for i in range(0, N):
        if i == 0 or i == N - 1:
            firstRow = list()
            for j in range(0, M):
                firstRow.append(1)
            room.append(firstRow)        
        else:
            for j in range(0, M):
                firstElement = 1
                lastElement = 1
                row = list()
                row.append(1)
                for k in range(0, M-2):
                    row.append(0)
                row.append(1)
            room.append(row)
    if num_exits is None:
        D = random.randint(1,6)
    else:
        D = num_exits 
    for i in range(0, D):
        doorCoordinateX = random.randint(1, N-2)
        doorCoordinateY = random.choice([0, M-1])
        if doorCoordinateX > 1 and doorCoordinateX < (N-2):
            doorCoordinateX_1 = doorCoordinateX + 1
            doorCoordinateY_1 = doorCoordinateY
            door1_tup = ((doorCoordinateX, doorCoordinateY), (doorCoordinateX_1, doorCoordinateY_1))
        else:
            door1_tup = ((doorCoordinateX, doorCoordinateY),)
        door2CoordinateX = random.choice([0, N-1])
        door2CoordinateY = random.randint(1, M-2)
        if door2CoordinateY > 1 and door2CoordinateY < (M - 2):
            door2CoordinateX_1 = door2CoordinateX
            door2CoordinateY_1 = door2CoordinateY + 1
            door2_tup = ((door2CoordinateX, door2CoordinateY), (door2CoordinateX_1, door2CoordinateY_1))
        else:
            door2_tup = ((door2CoordinateX, door2CoordinateY),)
        door_min = random.choice([door1_tup, door2_tup])
        #print(door_min)
        if len(door_min) > 1:
            room[door_min[0][0]][door_min[0][1]] = 4
            room[door_min[1][0]][door_min[1][1]] = 4
        else:
            room[door_min[0][0]][door_min[0][1]] = 4
    return room

In [16]:
# Randomize Map Method 3
'''
Randomly generate NxM array (25 > N > 5, 25 > M > 5) 
where the room is bounded by walls and all of the inner space is empty.
However, random columns of walls exist that jut out from the walls. 
A person would still need to be able to get to an exist.
'''
def generate_room_with_inner_walls(N, M, num_exits = None):
    room = []
    # N = 9
    # M = 8
    for i in range(0, N):
        if i == 0 or i == N - 1:
            firstRow = list()
            for j in range(0, M):
                firstRow.append(1)
            room.append(firstRow)        
        else:
            for j in range(0, M):
                firstElement = 1
                lastElement = 1
                row = list()
                row.append(1)
                for k in range(0, M-2):
                    row.append(0)
                row.append(1)
            room.append(row)
    if num_exits is None:
        D = random.randint(1,6)
    else:
        D = num_exits
    for i in range(0, D):
        doorCoordinateX = random.randint(1, N-2)
        doorCoordinateY = random.choice([0, M-1])
        door2CoordinateX = random.choice([0, N-1])
        door2CoordinateY = random.randint(1, M-2)
        door = (doorCoordinateX, doorCoordinateY)
        door2 = (door2CoordinateX, door2CoordinateY)
        door_min = random.choice([door, door2])
        room[door_min[0]][door_min[1]] = 4
    mansa = [(ix,iy) for ix, row in enumerate(room) for iy, i in enumerate(row) if i == 1]
    corners = [(0,0), (0, M-1), (N-1, 0), (N-1, M-1)]
    for i in range(0, random.randint(0, 4)):
        column = random.choice(mansa)
        if column not in corners:
            print(column)
            if column[1] == 0 and column[0] > 0:
                num = random.randint(1, M - 3) # M - 3 because we want to leave some empty space, don't want to 
                # completely divide room
                for j in range(1, num + 1):
                    if room[column[0]][j] == 0:
                        room[column[0]][j] = 1
            elif column[0] == 0 and column[1] > 0:
                num = random.randint(1, N - 3)
                for j in range(1, num + 1):
                    if room[j][column[1]] == 0:
                        room[j][column[1]] = 1
            elif column[0] == (N - 1) and column[1] > 0:
                num = random.randint(1, N - 3)
                for j in range(1, num + 1):
                    if room[j][column[1]] == 0:
                        room[j][column[1]] = 1
            else:
                num = random.randint(1, M - 3)
                for j in range(1, num + 1):
                    if room[column[0]][j] == 0:
                        room[column[0]][j] = 1  
    return room

In [17]:
# Heuristic Algorithm for Determining Map Fitness based on the Simulation's Statistics Dictionary
def get_score(simulation_dict):
    room_enum = {'generate_room_escape_doors': 1, 'generate_room_consecutive_doors': 2, 'generate_room_with_inner_walls': 0}
    score = 0
    score += simulation_dict['collisions']/5
    score += simulation_dict['pathSumAggregate']/simulation_dict['personCount']
    score -= simulation_dict['hazards']/10
    score += room_enum['generate_room_with_inner_walls']
    score += (simulation_dict['deaths'] * 5)
    return score

In [19]:
# Testing Cell: Simulate on Random Room
N = random.randint(5, 15)
M = random.randint(5, 15)
num_exits = math.ceil((N * M)/25)

room = generate_room_with_inner_walls(N, M, num_exits)
room = Room(room)
room_5_empty_spots = room.get_element_coordinates_of_map(RoomElement.EMPTY)
room_5_empty_spots_subset = random_elements_of_list(room_5_empty_spots, len(room_5_empty_spots), num_exits * 3)
print('Character Spots: ' + str(room_5_empty_spots_subset))

simulation5 = EvacuationSimulation(room, room_5_empty_spots_subset)
davo = simulation5.simulate_with_hazards()
print(davo)
print(get_score(davo))

(0, 4)
(0, 3)
Character Spots: [<2,1>, <3,1>, <1,5>, <3,4>, <4,3>, <3,6>, <4,5>, <3,7>, <4,1>]
Initialize simulation
Time: 0
[[   1    1    1    1    1    1    1    1    1]
 [   4    0    0    1    1 9999    0    0    1]
 [   1 9999    0    1    0    0    0    0    1]
 [   1 9999    0    1 9999    0 9999 9999    1]
 [   1 9999    0 9999    0 9999    0    0    1]
 [   1    4    1    4    1    1    1    1    1]]
Hazards: [<1,6>]
Dictionary: {<1,1>: [Person: @<1,1>, Path: [<1,1>, <1,0>], Exited? False], <4,1>: [Person: @<4,1>, Path: [<4,1>, <5,1>], Exited? False], <2,5>: [Person: @<2,5>, Path: [<2,5>, <3,5>, <4,5>, <4,4>, <4,3>, <5,3>], Exited? False], <4,4>: [Person: @<4,4>, Path: [<4,4>, <4,3>, <5,3>], Exited? False], <5,3>: [Person: @<5,3>, Path: [<5,3>], Exited? False], <4,6>: [Person: @<4,6>, Path: [<4,6>, <4,5>, <4,4>, <4,3>, <5,3>], Exited? False], <4,7>: [Person: @<4,7>, Path: [<4,7>, <4,6>, <4,5>, <4,4>, <4,3>, <5,3>], Exited? False], <5,1>: [Person: @<5,1>, Path: [<5,1>], Exited

In [20]:
# Testing Cell: Room Generation
for i in range(0, 10):
#     generated_room = generate_room_escape_doors(15, 15, 5)
    generated_room = generate_room_consecutive_doors(15, 15, 5)
#     generated_room = generate_room_with_inner_walls(15, 15, 5)
#     generated_room = room_1_map

    room = Room(generated_room)
    print(room.return_current_map_copy())
    print("~~~~~~~~~~~~~~~~~~~")

[[1 4 1 1 1 1 1 1 4 4 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 4]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 4]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 1 1 1 4 4 1 1 1 1 1 1 1 4 1]]
~~~~~~~~~~~~~~~~~~~
[[1 1 1 1 1 1 1 1 1 1 1 4 4 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 4]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 4]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [4 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 1 1 1 1 1 1 1 1 1 

In [24]:
# Analysis: Statistics Collection
'''
Parameter:

3 Room Types: (M, N, D)
Character Spots: proportions (0.2, 0.4, ...)

5 Params: map type, M (width), N (height), D (doors), P (proportion of occupied space aka character size)

For each map type:
    Try every combination of M, N, D, P
        Repeat 5x (different people distribution)
        Collect the stats for every time with this combination
        Calculate aggregate overall stats
        Store this information against the values (dict[M, N, D, P] = stats)

Revisit and look at overall score, sort

Possible parameter:
hazard spawn/expansion rates (S, E)
    
'''
mMax = 10
nMax = 10
dMax = 4

map_dict = {}

for M in range(5, mMax):
    for N in range(M, nMax):
        for D in range(1, dMax):
            stat_dict_list = []
            for P in [0.1, 0.25, 0.5, 0.75, 0.9]:
                print("M: " + str(M))
                print("N: " + str(N))
                print("D: " + str(D))
                print("P: " + str(P))
                
                room = generate_room_with_inner_walls(N, M, D)
                room = Room(room)
                room_empty_spots = room.get_element_coordinates_of_map(RoomElement.EMPTY)
                room_empty_spots_subset = random_elements_of_list(room_empty_spots, len(room_empty_spots), math.ceil(len(room_empty_spots)*P))
                print('Character Spots: ' + str(room_empty_spots_subset))
                simulation_current = EvacuationSimulation(room, room_empty_spots_subset)
                davo = simulation_current.simulate_with_hazards()
                
                print("Stats: " + str(davo))
                print("Score: " + str(get_score(davo)))
                stat_dict_list.append(davo)
                #
                
            bestSimulation = None
            bestScore = -999
            for stat in stat_dict_list:
                score = get_score(stat)
                if score > bestScore:
                    bestSimulation = stat
            map_dict[(M, N, D)] = bestSimulation

print("Final")
print(map_dict)

M: 5
N: 5
D: 1
P: 0.1
Character Spots: [<3,3>]
Initialize simulation
Time: 0
[[   1    4    1    1    1]
 [   1    0    0    0    1]
 [   1    0    0    0    1]
 [   1    0    0 9999    1]
 [   1    1    1    1    1]]
Hazards: []
Dictionary: {<2,3>: [Person: @<3,3>, Path: [<2,3>, <1,3>, <1,2>, <1,1>, <0,1>], Exited? False]}
Time: 1
Coordinates: [<2,3>]
[[   1    4    1    1    1]
 [   1    0    0    0    1]
 [   1    0    0 9999    1]
 [   1    0    0    0    1]
 [   1    1    1    1    1]]
Hazards: []
Dictionary: {<1,3>: [Person: @<2,3>, Path: [<2,3>, <1,3>, <1,2>, <1,1>, <0,1>], Exited? False]}
Time: 2
Coordinates: [<1,3>]
[[   1    4    1    1    1]
 [   1    0    0 9999    1]
 [   1    0    0    0    1]
 [   1    0    0    0    1]
 [   1    1    1    1    1]]
Hazards: []
Dictionary: {<1,2>: [Person: @<1,2>, Path: [<1,2>, <1,1>, <0,1>], Exited? False]}
Time: 3
Coordinates: [<1,2>]
[[   1    4    1    1    1]
 [   1    0 9999    0    1]
 [   1    0    0    0    1]
 [   1    0    0   

Dictionary: {<1,0>: [Person: @<1,0>, Path: [<1,0>], Exited? False], <4,3>: [Person: @<4,3>, Path: [<4,3>], Exited? False], <2,1>: [Person: @<2,1>, Path: [<2,1>, <1,1>, <1,0>], Exited? False], <1,1>: [Person: @<1,1>, Path: [<1,1>, <1,0>], Exited? False], <1,2>: [Person: @<1,2>, Path: [<1,2>, <1,1>, <1,0>], Exited? False], <1,3>: [Person: @<1,3>, Path: [<1,3>, <1,4>], Exited? False], <2,2>: [Person: @<2,2>, Path: [<2,2>, <1,2>, <1,1>, <1,0>], Exited? False], <1,4>: [Person: @<1,4>, Path: [<1,4>], Exited? False]}
Time: 1
Coordinates: [<1,0>, <4,3>, <2,1>, <1,1>, <1,1>, <1,2>, <1,3>, <2,2>, <1,4>]
[[   1    1    1    1    1]
 [9999 9999 9999 9999 9999]
 [   1 9999 9999    0    1]
 [   1    0    0    3    1]
 [   1    1    1 9999    1]]
Hazards: [<3,3>, <1,2>]
Dictionary: {<1,0>: [Person: @<1,0>, Path: [<1,0>], Exited? False], <4,3>: [Person: @<4,3>, Path: [<4,3>], Exited? True], <1,1>: [Person: @<1,1>, Path: [<1,1>, <1,0>], Exited? False], <1,4>: [Person: @<1,4>, Path: [<1,4>], Exited? Tru

[[   1    1    1 9999    1]
 [   1    0    0 9999    4]
 [   1    1    1    0    1]
 [   1    3    0    0    1]
 [   1    3    3    0    1]
 [   1    1    1    1    1]]
Hazards: [<3,1>, <4,1>, <4,2>]
Dictionary: {<0,3>: [Person: @<0,3>, Path: [<0,3>], Exited? True]}
Stats: {'time': 4, 'personCount': 3, 'collisions': 0, 'hazards': 3, 'pathSumAggregate': 8, 'deaths': 0, 'error': False, 'map': <__main__.Room object at 0x7fc9dffa5970>}
Score: 2.3666666666666667
M: 5
N: 6
D: 2
P: 0.5
(3, 0)
Character Spots: [<4,1>, <4,2>, <3,2>, <2,1>, <1,3>, <2,3>]
Initialize simulation
Time: 0
[[   1    1    1    1    1]
 [   1    0    0 9999    1]
 [   1 9999    0 9999    4]
 [   1    1 9999    0    1]
 [   1 9999 9999    0    1]
 [   1    4    1    1    1]]
Hazards: []
Dictionary: {<5,1>: [Person: @<4,1>, Path: [<5,1>], Exited? False], <4,1>: [Person: @<4,2>, Path: [<4,1>, <5,1>], Exited? False], <2,2>: [Person: @<2,1>, Path: [<2,2>, <2,3>, <2,4>], Exited? False], <2,3>: [Person: @<1,3>, Path: [<2,3>, <

Initialize simulation
Time: 0
[[   1    1    1    4    1]
 [   1    1    0    0    1]
 [   1    0    0    0    1]
 [   1    0    0    0    4]
 [   1 9999 9999    0    1]
 [   1    1    1    1    1]]
Hazards: []
Dictionary: {<3,2>: [Person: @<4,2>, Path: [<3,2>, <3,3>, <3,4>], Exited? False], <3,1>: [Person: @<4,1>, Path: [<3,1>, <3,2>, <3,3>, <3,4>], Exited? False]}
Time: 1
Coordinates: [<3,2>, <3,1>]
[[   1    1    1    4    1]
 [   1    1    0    0    1]
 [   1    0    0    0    1]
 [   1 9999 9999    0    4]
 [   1    0    0    0    1]
 [   1    1    1    1    1]]
Hazards: []
Dictionary: {<3,3>: [Person: @<3,3>, Path: [<3,3>, <3,4>], Exited? False], <3,2>: [Person: @<3,2>, Path: [<3,2>, <3,3>, <3,4>], Exited? False]}
Time: 2
Coordinates: [<3,3>, <3,2>]
[[   1    1    1    4    1]
 [   1    1    0    0    1]
 [   1    0    0    0    1]
 [   1    0 9999 9999    4]
 [   1    0    0    0    1]
 [   1    1    1    1    1]]
Hazards: [<2,2>]
Dictionary: {<3,4>: [Person: @<3,4>, Path: [<3,4

Dictionary: {<3,2>: [Person: @<3,2>, Path: [<3,2>, <3,1>, <3,0>], Exited? False], <3,1>: [Person: @<3,1>, Path: [<3,1>, <3,0>], Exited? False], <3,0>: [Person: @<3,0>, Path: [<3,0>], Exited? False]}
Time: 4
Coordinates: [<3,2>, <3,1>, <3,0>, <3,0>]
[[   1    1    1    1    1]
 [   1    0    0    0    1]
 [   1    0    0    0    1]
 [9999 9999 9999    0    1]
 [   1    1    1    0    1]
 [   1    0    0    0    1]
 [   1    1    1    1    1]]
Hazards: [<3,3>]
Dictionary: {<3,1>: [Person: @<3,1>, Path: [<3,1>, <3,0>], Exited? False], <3,0>: [Person: @<3,0>, Path: [<3,0>], Exited? True]}
Time: 5
Coordinates: [<3,1>, <3,0>, <3,0>, <3,0>]
[[   1    1    1    1    1]
 [   1    0    0    0    1]
 [   1    0    0    0    1]
 [9999 9999    0    3    1]
 [   1    1    1    0    1]
 [   1    0    0    0    1]
 [   1    1    1    1    1]]
Hazards: [<3,3>, <3,1>]
Dictionary: {<3,0>: [Person: @<3,0>, Path: [<3,0>], Exited? True]}
Stats: {'time': 5, 'personCount': 4, 'collisions': 0, 'hazards': 2, 'p

...

Dictionary: {<7,8>: [Person: @<7,8>, Path: [<7,8>], Exited? True]}
Stats: {'time': 17, 'personCount': 39, 'collisions': 0, 'hazards': 8, 'pathSumAggregate': 260, 'deaths': 0, 'error': False, 'map': <__main__.Room object at 0x7fc9e34808b0>}
Score: 5.866666666666667
M: 9
N: 9
D: 3
P: 0.1
(0, 6)
(1, 8)
(8, 3)
Character Spots: [<7,1>, <6,3>, <7,3>, <3,2>]
Initialize simulation
Time: 0
[[   1    4    1    1    1    1    1    1    1]
 [   4    1    1    1    0    0    1    0    1]
 [   1    0    0    1    0    0    1    0    1]
 [   1    0 9999    1    0    0    1    0    1]
 [   4    0    0    1    0    0    1    0    1]
 [   1    0    0    0    0    0    1    0    1]
 [   1    0    0 9999    0    0    1    0    1]
 [   1 9999    0 9999    0    0    0    0    1]
 [   1    1    1    1    1    1    1    1    1]]
Hazards: []
Dictionary: {<6,1>: [Person: @<7,1>, Path: [<6,1>, <5,1>, <4,1>, <4,0>], Exited? False], <5,3>: [Person: @<6,3>, Path: [<5,3>, <5,2>, <4,2>, <4,1>, <4,0>], Exited? False],

Dictionary: {<0,3>: [Person: @<0,3>, Path: [<0,3>], Exited? False], <5,7>: [Person: @<5,7>, Path: [<5,7>, <5,8>], Exited? False], <6,1>: [Person: @<6,1>, Path: [<6,1>, <6,0>], Exited? False], <5,8>: [Person: @<5,7>, Path: [<5,8>], Exited? True], <6,0>: [Person: @<6,0>, Path: [<6,0>], Exited? True], <5,6>: [Person: @<5,6>, Path: [<5,6>, <5,7>, <5,8>], Exited? False], <1,4>: [Person: @<1,4>, Path: [<1,4>, <1,3>, <0,3>], Exited? False], <1,3>: [Person: @<1,3>, Path: [<1,3>, <0,3>], Exited? False]}
Time: 3
Coordinates: [<0,3>, <5,7>, <5,7>, <6,1>, <5,7>, <0,3>, <6,0>, <5,6>, <0,3>, <1,4>, <6,1>, <1,3>]
[[   1    1    1 9999    1    1    1    1    1]
 [   1    1    1 9999 9999    0    0    0    1]
 [   1    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    1]
 [   1    3    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0 9999 9999    4]
 [9999 9999    0    0    0    0    0    0    1]
 [   1    1    0    0    0    0    0    0    1]
 [   1    

Dictionary: {<3,8>: [Person: @<3,8>, Path: [<3,8>], Exited? False], <1,1>: [Person: @<1,1>, Path: [<1,1>, <1,0>], Exited? False], <1,2>: [Person: @<1,2>, Path: [<1,2>, <1,1>, <1,0>], Exited? False], <1,0>: [Person: @<1,0>, Path: [<1,0>], Exited? False], <3,7>: [Person: @<3,7>, Path: [<3,7>, <3,8>], Exited? False], <2,1>: [Person: @<2,1>, Path: [<2,1>, <1,1>, <1,0>], Exited? False], <7,8>: [Person: @<7,8>, Path: [<7,8>], Exited? False], <7,7>: [Person: @<7,7>, Path: [<7,7>, <7,8>], Exited? False]}
Time: 5
Coordinates: [<3,8>, <3,8>, <1,1>, <3,8>, <1,2>, <3,8>, <1,2>, <1,0>, <1,2>, <3,7>, <2,1>, <3,8>, <1,0>, <1,0>, <7,8>, <1,2>, <3,8>, <1,1>, <3,8>, <3,8>, <3,8>, <1,0>, <1,0>, <7,7>]
[[   1    1    1    1    1    1    1    1    1]
 [9999 9999 9999    0    0    0    0    1    1]
 [   1 9999    0    0    0    0    3    0    1]
 [   1    0    0    0    0    0    0 9999 9999]
 [   1    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    1]
 [   1    0    0    

Dictionary: {<5,2>: [Person: @<5,2>, Path: [<5,2>, <6,2>, <7,2>, <8,2>], Exited? False], <7,6>: [Person: @<7,6>, Path: [<7,6>, <8,6>], Exited? False], <4,1>: [Person: @<4,1>, Path: [<4,1>, <5,1>, <6,1>, <7,1>, <7,2>, <8,2>], Exited? False], <8,6>: [Person: @<8,6>, Path: [<8,6>], Exited? False], <6,7>: [Person: @<6,7>, Path: [<6,7>, <7,7>, <7,8>], Exited? False], <8,2>: [Person: @<8,2>, Path: [<8,2>], Exited? True], <4,3>: [Person: @<4,3>, Path: [<4,3>, <5,3>, <6,3>, <7,3>, <7,2>, <8,2>], Exited? False], <7,2>: [Person: @<7,2>, Path: [<7,2>, <8,2>], Exited? False], <7,7>: [Person: @<7,7>, Path: [<7,7>, <7,8>], Exited? False], <7,1>: [Person: @<7,1>, Path: [<7,1>, <7,2>, <8,2>], Exited? False], <4,4>: [Person: @<4,4>, Path: [<4,4>, <5,4>, <6,4>, <7,4>, <7,3>, <7,2>, <8,2>], Exited? False], <7,5>: [Person: @<7,5>, Path: [<7,5>, <7,6>, <8,6>], Exited? False], <4,2>: [Person: @<4,2>, Path: [<4,2>, <5,2>, <6,2>, <7,2>, <8,2>], Exited? False], <6,1>: [Person: @<6,1>, Path: [<6,1>, <7,1>, <7,2

Dictionary: {<8,2>: [Person: @<8,2>, Path: [<8,2>], Exited? True], <8,6>: [Person: @<8,6>, Path: [<8,6>], Exited? True], <7,8>: [Person: @<7,8>, Path: [<7,8>], Exited? True], <7,2>: [Person: @<7,2>, Path: [<7,2>, <8,2>], Exited? False]}
Time: 8
Coordinates: [<8,2>, <8,6>, <8,2>, <8,6>, <7,8>, <8,2>, <8,2>, <8,2>, <7,8>, <8,2>, <7,2>, <8,2>, <8,6>, <8,6>, <8,2>, <8,2>, <8,2>, <8,6>, <8,2>, <8,2>, <7,8>, <8,2>, <8,6>, <8,6>, <7,8>, <8,2>, <7,8>, <8,2>, <8,2>, <8,6>, <8,6>, <8,2>, <8,6>, <7,8>, <8,2>, <8,6>, <8,6>]
[[   1    1    1    1    1    1    1    1    1]
 [   1    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    1]
 [   1    0    0    0    0    0    0    0    1]
 [   1    0    0    3    0    3    0    0    1]
 [   1    0 9999    3    0    0    0    0 9999]
 [   1    1 9999    1    1    1 9999    1    1]]
Hazards: [<7,3>, <6,3>, <6,5>, <2,5>]
Dictionary: